In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [8]:
PATH = "data/dogscats/"
sz=224
batch_size=64

In [13]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.applications.resnet50 import preprocess_input

In [14]:
train_data_dir = f'{PATH}train'
validation_data_dir = f'{PATH}valid'

In [19]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(sz, sz),
                batch_size=batch_size, class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_data_dir, shuffle=False,
                    target_size=(sz, sz), batch_size=batch_size, class_mode='binary')

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [21]:
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [18]:
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=3, workers=4,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/3
359/359 [==============================] - 124s 346ms/step - loss: 0.1968 - acc: 0.9485 - val_loss: 0.0582 - val_acc: 0.9859
Epoch 2/3
359/359 [==============================] - 115s 322ms/step - loss: 0.0792 - acc: 0.9714 - val_loss: 0.0615 - val_acc: 0.9840
Epoch 3/3
359/359 [==============================] - 113s 315ms/step - loss: 0.0673 - acc: 0.9766 - val_loss: 0.0714 - val_acc: 0.9830
CPU times: user 17min, sys: 1min 10s, total: 18min 11s
Wall time: 5min 53s


In [23]:
split_at = 140
for layer in model.layers[:split_at]: 
    layer.trainable = False
for layer in model.layers[split_at:]: 
    layer.trainable = True
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=1, workers=3,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/1
359/359 [==============================] - 145s 404ms/step - loss: 0.0739 - acc: 0.9757 - val_loss: 0.3055 - val_acc: 0.9664
CPU times: user 6min 2s, sys: 34.2 s, total: 6min 36s
Wall time: 2min 26s


In [26]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation